In [2]:
import numpy as np
import pandas as pd

In [7]:
orders = pd.read_csv("/home/ksn38/data/transaction_data.csv")[['household_key', 'PRODUCT_ID', 'QUANTITY']]
orders.rename(columns={'household_key': 'user_id', 'PRODUCT_ID': 'item_id', 'QUANTITY': 'quantity'}, inplace=True)
orders.to_csv('/home/ksn38/data/orders.csv', header=False)

In [8]:
pd.read_csv('/home/ksn38/data/orders.csv')

,0,2375,1004906,1
0,1,2375,1033142,1
1,2,2375,1036325,1
2,3,2375,1082185,1
3,4,2375,8160430,1
4,5,2375,826249,2
...,...,...,...,...
2595726,2595727,1598,92130,1
2595727,2595728,1598,114102,1
2595728,2595729,1598,133449,1
2595729,2595730,1598,6923644,1


In [ ]:
#\copy orders FROM '/home/ksn38/data/orders.csv' DELIMITER ',' CSV;

In [1]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [10]:
engine = create_engine('postgresql://ksn38:1@localhost:5432/sparkdb', echo=True)
Base = declarative_base()

In [19]:
class vacancy(Base):
    __tablename__ = 'vacancy'
    id = Column(Integer, primary_key=True, unique=True, autoincrement=True)
    name = Column(String)
    salary = Column(Integer)
    link = Column(String)

    def __init__(self, name, salary, link):
        self.name = name
        self.salary = salary
        self.link = link


def creator():
    Base.metadata.create_all(engine)

In [21]:
def addition(engine):
    Session = sessionmaker(bind=engine)
    session = Session()

    session.add_all([vacancy(i, j, k) for i, j, k in zip('bob', '100', 'link')])

    session.commit()
    session.close()

In [22]:
# creator()
addition(engine)

2021-05-21 18:21:06,410 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-05-21 18:21:06,414 INFO sqlalchemy.engine.base.Engine INSERT INTO vacancy (name, salary, link) VALUES (%(name)s, %(salary)s, %(link)s) RETURNING vacancy.id
2021-05-21 18:21:06,415 INFO sqlalchemy.engine.base.Engine {'name': 'b', 'salary': '1', 'link': 'l'}
2021-05-21 18:21:06,417 INFO sqlalchemy.engine.base.Engine INSERT INTO vacancy (name, salary, link) VALUES (%(name)s, %(salary)s, %(link)s) RETURNING vacancy.id
2021-05-21 18:21:06,422 INFO sqlalchemy.engine.base.Engine {'name': 'o', 'salary': '0', 'link': 'i'}
2021-05-21 18:21:06,424 INFO sqlalchemy.engine.base.Engine INSERT INTO vacancy (name, salary, link) VALUES (%(name)s, %(salary)s, %(link)s) RETURNING vacancy.id
2021-05-21 18:21:06,425 INFO sqlalchemy.engine.base.Engine {'name': 'b', 'salary': '0', 'link': 'n'}
2021-05-21 18:21:06,435 INFO sqlalchemy.engine.base.Engine COMMIT


In [36]:
def comparator(engine, num):
    Session = sessionmaker(bind=engine)
    session = Session()

    for i in session.query(vacancy).filter(vacancy.salary > num):
        print(i.name, i.salary)

    session.commit()
    session.close()


# creator()
# addition(engine)
comparator(engine, 0)

2021-05-21 18:33:29,501 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-05-21 18:33:29,504 INFO sqlalchemy.engine.base.Engine SELECT vacancy.id AS vacancy_id, vacancy.name AS vacancy_name, vacancy.salary AS vacancy_salary, vacancy.link AS vacancy_link 
FROM vacancy 
WHERE vacancy.salary > %(salary_1)s
2021-05-21 18:33:29,512 INFO sqlalchemy.engine.base.Engine {'salary_1': 0}
b 1
2021-05-21 18:33:29,521 INFO sqlalchemy.engine.base.Engine COMMIT


In [4]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine

# automap base
Base = automap_base()

# pre-declare User for the 'user' table
class Orders(Base):
    __tablename__ = 'orders'

    # override schema elements like Columns
    id = Column(Integer, primary_key=True, unique=True, autoincrement=True)
    user_id = Column('user_id', Integer)
    item_id = Column('item_id', Integer)
    quantity = Column('quantity', Integer)


# reflect
engine = create_engine('postgresql://ksn38:1@localhost:5432/sparkdb', echo=True)
Base.prepare(engine, reflect=True)

2021-05-21 19:26:37,244 INFO sqlalchemy.engine.base.Engine select version()
2021-05-21 19:26:37,258 INFO sqlalchemy.engine.base.Engine {}
2021-05-21 19:26:37,281 INFO sqlalchemy.engine.base.Engine select current_schema()
2021-05-21 19:26:37,296 INFO sqlalchemy.engine.base.Engine {}
2021-05-21 19:26:37,316 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-05-21 19:26:37,322 INFO sqlalchemy.engine.base.Engine {}
2021-05-21 19:26:37,325 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-05-21 19:26:37,326 INFO sqlalchemy.engine.base.Engine {}
2021-05-21 19:26:37,329 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2021-05-21 19:26:37,331 INFO sqlalchemy.engine.base.Engine {}
2021-05-21 19:26:37,334 INFO sqlalchemy.engine.base.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2021-0

2021-05-21 19:26:37,540 INFO sqlalchemy.engine.base.Engine {'table_name': 'vacancy'}
2021-05-21 19:26:37,543 INFO sqlalchemy.engine.base.Engine 
            SELECT a.attname,
              pg_catalog.format_type(a.atttypid, a.atttypmod),
              (SELECT pg_catalog.pg_get_expr(d.adbin, d.adrelid)
                FROM pg_catalog.pg_attrdef d
               WHERE d.adrelid = a.attrelid AND d.adnum = a.attnum
               AND a.atthasdef)
              AS DEFAULT,
              a.attnotnull, a.attnum, a.attrelid as table_oid,
              pgd.description as comment,
              NULL as generated
            FROM pg_catalog.pg_attribute a
            LEFT JOIN pg_catalog.pg_description pgd ON (
                pgd.objoid = a.attrelid AND pgd.objsubid = a.attnum)
            WHERE a.attrelid = %(table_oid)s
            AND a.attnum > 0 AND NOT a.attisdropped
            ORDER BY a.attnum
        
2021-05-21 19:26:37,544 INFO sqlalchemy.engine.base.Engine {'table_oid': 17556}
2021-

In [5]:
Session = sessionmaker(bind=engine)
session = Session()

order1 = session.query(Orders).first()
print (order1.user_id, order1.item_id)

2021-05-21 19:26:43,927 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-05-21 19:26:43,931 INFO sqlalchemy.engine.base.Engine SELECT orders.id AS orders_id, orders.user_id AS orders_user_id, orders.item_id AS orders_item_id, orders.quantity AS orders_quantity 
FROM orders 
 LIMIT %(param_1)s
2021-05-21 19:26:43,933 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
2375 1004906


In [27]:
def selector(engine, num):
    Session = sessionmaker(bind=engine)
    session = Session()

    for i in session.query(Orders).filter(Orders.quantity > num):
        print(i.item_id)

    session.close()


selector(engine, 10000)

2021-05-21 19:23:10,726 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-05-21 19:23:10,737 INFO sqlalchemy.engine.base.Engine SELECT orders.id AS orders_id, orders.user_id AS orders_user_id, orders.item_id AS orders_item_id, orders.quantity AS orders_quantity 
FROM orders 
WHERE orders.quantity > %(quantity_1)s
2021-05-21 19:23:10,741 INFO sqlalchemy.engine.base.Engine {'quantity_1': 10000}
6534178
6534178
397896
6534166
6534178
6534178
6534178
6534178
6534178
5747420
6534178
6534178
397896
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534166
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6533889
6534178
6534178
6533889
6534178
6534178
397896
6534178
6534178
6534166
6534178
6534178
6534166
6534178
6533889
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166


6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6410462
6534178
6533889
6534178
6533889
6534166
6533889
6534178
6534178
6533889
6534178
6534166
6534178
6534178
6534178
6534166
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6533889
6534166
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6544236
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178


6534178
6534178
6534178
6533889
6534166
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6534178
5703832
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6544236
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6533889
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534166
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6533889
6534178
1426702
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178


6534178
6534178
6534166
6534178
1404121
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6533889
6534166
6534178
6533889
6534166
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534178
1426702
1404121
5716076
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6544236
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6544236
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
1404121
707683
480014
6534178
6544236
6534178
65

6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
397896
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534166
6534178
6534178
6533889
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6533889
6533889
6533889
6534178
6534178
6534166
6534178
6544236
6534178
6533889
6534178
6534178
6534178
6534178
420647
6533889
6534178
6534178
6534166
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534166
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6544236
6534178
6534178
1404121
6534178
6534178
1404121
65

6534166
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534166
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6533889
6534178
6534178
6534178
6534178
6534178
6533889
6533889
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534166
1404121
6534178
6534178
6534178
6534178
6533889
6534178
6534166
6533889
6534166
6534178
6534178
6534178
6534178
6534166
6534178
6534178


6534178
6534178
6534178
6534178
6533889
6534178
6533889
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6533889
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6533889
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
1404121
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534166
6534178
6533889


6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
545926
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6544236
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534166
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
397896
6534178
6534178
6534178
6533889
6533889
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6534166
6534178
6534178
6534178
6534178
6534178
6534178
6534178
6533889
6534178
6534178
6533889
65

In [1]:
from kafka import KafkaProducer
producer = KafkaProducer(bootstrap_servers='localhost:9092')
producer.send('als_kafka2', b'{"user_id":2375}')